In [1]:
from service.MatchSys.match_sys import MatchSys
from service import config
config.initialize()
ms = MatchSys(
        name='teat_sys',
        ltp_model_path=r'D:\Code\MyLongTimeProject\A\QQ-Bot-And-Tool\data\LtpModel',
        database_uri='sqlite:///data/db.sqlite3',
        text_vec_model_path=r'D:\Code\MyLongTimeProject\A\QQ-Bot-And-Tool\data\model.pkl'
    )

1713847036446646272
Loading weights from local directory
# jionlp - 微信公众号: JioNLP  Github: `https://github.com/dongrixinyu/JioNLP`.
# jiojio - `http://www.jionlp.com/jionlp_online/cws_pos` is available for online trial.


In [2]:
from service.MatchSys.trainers import Trainer
from service.MatchSys.utils import print_progress_bar


class QATrainer(Trainer):
    """
    Allows a chat bot to be trained using a list of strings
    where the list represents a conversation.
    """

    def train(self, conversation, **kwargs):
        """
        {Q:[A1,A2...]}
        Train the chat bot based on the provided list of
        statements that represents a single conversation.
        """
        source = kwargs.get('source', 'TRAIN_DATA')
        conversation_text = kwargs.get('conversation', 'TRAIN_DATA')
        statements_to_create = []
        previous_id = 0
        next_id = 0
        input_statements = self.chatbot.message_adapter.process_list(list(conversation.keys()),**kwargs)
        for index,input_statement in enumerate(input_statements):
                if self.show_training_progress:
                    print_progress_bar(
                        'QA Trainer',
                        index + 1, len(conversation)
                    )
                try:
                    statements_to_create.append(input_statement)
                    kwargs['id'] = next_id
                    kwargs['type_of'] = 'A'
                    kwargs['persona'] = 'bot:'+self.chatbot.name
                    for statement in  self.chatbot.message_adapter.process_list(conversation[input_statement.text],**kwargs):

                        statements_to_create.append(statement)
                except Exception as e:
                    print(statement)
        self.chatbot.storage.create_many(statements_to_create)
        self.chatbot.docvector_tool.train(statements_to_create)


import pandas as pd
map = {}
data = pd.read_excel(r"D:\temp\Downloads\傲娇系二次元bot词库5千词V1.2 - Test.xlsx",header=None, sheet_name=0)
for index,row in data.iterrows():
    if row[0] in map:
        map[row[0]].append(row[1])
    else:
        map[row[0]] = [row[1]]

kwargs = {}
statements_to_create = []
previous_id = 0
next_id = 0
input_statements = ms.message_adapter.process_list(list(map.keys()),**kwargs)
for index,input_statement in enumerate(input_statements):
    statements_to_create.append(input_statement)
    kwargs['id'] = next_id
    kwargs['type_of'] = 'A'
    kwargs['persona'] = 'bot:'+ms.name
    for statement in  ms.message_adapter.process_list(map[input_statement.text],**kwargs):
        statements_to_create.append(statement)

ms.storage.create_many(statements_to_create)
 

In [3]:
tokenized = []
from gensim.models.doc2vec import TaggedDocument
for statement in statements_to_create:
    if statement.id > 0:
        tokenized.append(TaggedDocument(statement.search_text.split(' '),tags=[statement.id,statement.text]))

In [6]:
tokenized = tokenized[:5]

In [10]:
from gensim.models import Doc2Vec
model = Doc2Vec(documents=tokenized,epochs=50)
ms.docvector_tool.model.train(tokenized,total_examples=ms.docvector_tool.model.corpus_count,epochs=10)

MemoryError: 

In [ ]:
trainer = QATrainer(ms)
trainer.train(map)

In [ ]:
trainer.train(map)

In [ ]:
res = ms.get_response('你')

In [ ]:
res[0].id